### Library import and code configuration

In [1]:
import numpy as np
import pandas as pd
import geocoder

In [2]:
pd.options.display.float_format = '{:.6f}'.format

Geocoder

In [3]:
def geoc(loc):
    g = geocoder.google(loc)
    g = g.json
    print (loc)
    if len(g) != 2 or g is None:
        g = geocoder.arcgis(loc)
        g = g.json

        if len(g) != 2 or g is None:
            g = geocoder.geocodefarm(loc)
            g = g.json

            if len(g) != 2 or g is None:
                g = geocoder.osm(loc)
                g = g.json

                if len(g) != 2 or g is None:
                    g = geocoder.yandex(loc)
                    g = g.json
    else:
        g = None
    
    print(g)
    return g

### Dataframe

In [7]:
fileIn = '/media/dedsresende/BackUpDeds/Github_UB/CapstoneProject/Feelings/kagle_capstone_project/presentacion/data/tweets_public_decode2.csv'
fileOut = '/media/dedsresende/BackUpDeds/Github_UB/CapstoneProject/Feelings/kagle_capstone_project/presentacion/data/tweets_public_clean.csv'

In [9]:
df = pd.read_csv(fileIn, encoding = 'utf-8')
df.head()

,tweet_id,airline_sentiment,is_reply,reply_count,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,users,hashtags
0,942743012337122944.000000,positive,False,0,0,"""Los pilotos de Ryanair desconvocan la huelga ...",NaN,Mon Dec 18 13:07:04 +0000 2017,NaN,Dublin,NaN,NaN
1,926857871916184064.000000,positive,True,0,0,"@Iberia @lavecinarubia Si ,por favor las decla...",NaN,Sat Nov 04 17:05:11 +0000 2017,NaN,NaN,Iberia|lavecinarubia,rubijarena
2,936979305720091008.000000,neutral,True,0,0,@Iberia Me dirías por favor que costo tiene?,NaN,Sat Dec 02 15:24:09 +0000 2017,NaN,NaN,Iberia,NaN
3,943983853802328064.000000,negative,True,0,0,"@SupermanlopezN @Iberia @giroditalia Champion,...",NaN,Thu Dec 21 23:17:43 +0000 2017,NaN,Central Time (US & Canada),SupermanlopezN|Iberia|giroditalia,NaN
4,938207464457212032.000000,negative,True,0,0,@SrtaFarrellDM @KLM @Iberia Eso de avianca es ...,NaN,Wed Dec 06 00:44:25 +0000 2017,NaN,Eastern Time (US & Canada),SrtaFarrellDM|KLM|Iberia,NaN


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7867 entries, 0 to 7866
Data columns (total 12 columns):
tweet_id             7867 non-null float64
airline_sentiment    7867 non-null object
is_reply             7867 non-null bool
reply_count          7867 non-null int64
retweet_count        7867 non-null int64
text                 7867 non-null object
tweet_coord          22 non-null object
tweet_created        7867 non-null object
tweet_location       439 non-null object
user_timezone        5119 non-null object
users                5259 non-null object
hashtags             1324 non-null object
dtypes: bool(1), float64(1), int64(2), object(8)
memory usage: 683.8+ KB


In [12]:
df['tweet_id'] = df['tweet_id'].astype('int64')
df['airline_sentiment'] = df['airline_sentiment'].astype('category')
df['retweet_count'] = df['retweet_count'].astype('int64')
df['text'] = df['text'].astype('str')
df['tweet_coord'] = df['tweet_coord'].astype('str')
df['tweet_created'] = pd.to_datetime(df['tweet_created'])
df['tweet_location'] = df['tweet_location'].astype('category')
df['user_timezone'] = df['user_timezone'].astype('category')

In [13]:
df['lat'] = df['tweet_coord'].apply(lambda x: np.nan if x == 'nan' else float((x.replace('[',' ').replace(']',' ').replace(' ','').split(','))[0]))
df['lon'] = df['tweet_coord'].apply(lambda x: np.nan if x == 'nan' else float((x.replace('[',' ').replace(']',' ').replace(' ','').split(','))[1]))

In [24]:
df['airline'] = df['users']

In [15]:
ttloc = {}

for loc in df[df['tweet_location'].notnull()]['tweet_location'].unique():
    g = geocoder.google(loc.split(',')[0])
    latlng = g.latlng
    
    if latlng is None:
        g = geocoder.arcgis(loc.split(',')[0])
        latlng = g.latlng

        if latlng is None:
            g = geocoder.geocodefarm(loc.split(',')[0])
            latlng = g.latlng

            if latlng is None:
                g = geocoder.osm(loc.split(',')[0])
                latlng = g.latlng

                if latlng is None:
                    g = geocoder.yandex(loc.split(',')[0])
                    latlng = g.latlng
    
    try:
        ttloc[loc] = {'lat': latlng[0], 'lng': latlng[1]}
        print(ttloc[loc])
        
    except Exception as e:
        print(e)

{'lat': 40.4167754, 'lng': -3.7037902}
{'lat': -34.459329999999966, 'lng': -58.91528999999997}
{'lat': 1.2058837, 'lng': -77.285787}
{'lat': 41.6488226, 'lng': -0.8890853}
{'lat': 40.63957000000005, 'lng': 22.93709000000007}
{'lat': 40.4167754, 'lng': -3.7037902}
{'lat': 43.529742, 'lng': 5.447426999999999}
{'lat': 19.4306069, 'lng': -99.0949899}
{'lat': 39.571480000000065, 'lng': 2.646940000000029}
{'lat': -33.3989812, 'lng': -70.5573124}
{'lat': 40.42028000000005, 'lng': -3.7057699999999727}
{'lat': 38.90482859999999, 'lng': -77.05239870000001}
{'lat': 43.3069814, 'lng': -2.9126022}
{'lat': 10.56737000000004, 'lng': -84.00272999999999}
{'lat': -34.6036844, 'lng': -58.3815591}
{'lat': 41.3120158, 'lng': 2.0959429}
{'lat': -1.428836290999925, 'lng': -78.77523855199996}
{'lat': 50.1109221, 'lng': 8.6821267}
{'lat': 41.388040000000046, 'lng': 2.1700100000000475}
{'lat': 10.3910485, 'lng': -75.4794257}
{'lat': 41.10934890000001, 'lng': 1.1400998}
{'lat': 40.5688791, 'lng': -3.5913939}
{'l

{'lat': 36.70601449999999, 'lng': 3.2281983}
{'lat': -34.5209468, 'lng': -58.4972596}
{'lat': 43.5579523, 'lng': -5.9246653}
{'lat': 40.055385, 'lng': 0.0641504}
{'lat': 51.1078852, 'lng': 17.0385376}
{'lat': -33.0153481, 'lng': -71.55002759999999}
{'lat': 48.71299000000005, 'lng': 2.3657100000000355}
{'lat': 38.72639000000004, 'lng': -9.149479999999926}
{'lat': 52.52000659999999, 'lng': 13.404954}
{'lat': 13.3387613, 'lng': -81.3729372}
{'lat': 41.7735409, 'lng': 12.2397117}
{'lat': 40.426210000000026, 'lng': -3.563989999999933}
{'lat': 40.7830603, 'lng': -73.9712488}
{'lat': 42.24059889999999, 'lng': -8.7207268}
{'lat': 50.503887, 'lng': 4.469936}
{'lat': 28.500821, 'lng': -13.8628367}
{'lat': 59.38292819999999, 'lng': 17.9644707}
{'lat': 43.0097384, 'lng': -7.5567582}
{'lat': -34.72511999999995, 'lng': -58.26012999999995}
{'lat': 42.4298846, 'lng': -8.6446202}
{'lat': 39.0200099, 'lng': 1.4821482}
{'lat': 20.687580000000025, 'lng': -103.35103999999995}
{'lat': 33.886917, 'lng': 9.53

In [16]:
usrloc = {}

for loc in df[df['user_timezone'].notnull()]['user_timezone'].unique():
    g = geocoder.google(loc.split(',')[0])
    latlng = g.latlng
    
    if latlng is None:
        g = geocoder.arcgis(loc.split(',')[0])
        latlng = g.latlng

        if latlng is None:
            g = geocoder.geocodefarm(loc.split(',')[0])
            latlng = g.latlng

            if latlng is None:
                g = geocoder.osm(loc.split(',')[0])
                latlng = g.latlng

                if latlng is None:
                    g = geocoder.yandex(loc.split(',')[0])
                    latlng = g.latlng
    
    try:
        usrloc[loc] = {'lat': latlng[0], 'lng': latlng[1]}
        print(usrloc[loc])
        
    except Exception as e:
        print(e)

{'lat': 53.3498053, 'lng': -6.2603097}
{'lat': 37.09024, 'lng': -95.712891}
{'lat': 37.51647579999999, 'lng': -82.8067126}
{'lat': 24.90560000000005, 'lng': 67.08220000000006}
{'lat': 54.80707000000007, 'lng': -128.27896999999996}
{'lat': 40.42028000000005, 'lng': -3.7057699999999727}
{'lat': 19.431940000000054, 'lng': -99.13314999999994}
{'lat': 10.4805937, 'lng': -66.90360629999999}
{'lat': 37.9838096, 'lng': 23.7275388}
{'lat': -34.6036844, 'lng': -58.3815591}
{'lat': -33.43721999999997, 'lng': -70.65001999999998}
{'lat': 74.72711633200004, 'lng': -41.34560937799995}
{'lat': 52.3702157, 'lng': 4.895167900000001}
{'lat': 52.52000659999999, 'lng': 13.404954}
{'lat': 45.807240000000036, 'lng': 15.967570000000023}
{'lat': 4.614960000000053, 'lng': -74.06940999999995}
{'lat': 48.856614, 'lng': 2.3522219}
{'lat': 51.5073509, 'lng': -0.1277583}
{'lat': 60.16985569999999, 'lng': 24.9383791}
{'lat': 28.2072168, 'lng': -177.3734926}
{'lat': 46.0569465, 'lng': 14.5057515}
{'lat': -34.6036844, 

In [17]:
df['tweet_location_coord'] = df['tweet_location'].map(ttloc)
df['user_timezone_coord'] = df['user_timezone'].map(usrloc)

In [18]:
df['tweet_location_coord'] = df['tweet_location_coord'].astype('object')
df['user_timezone_coord'] = df['user_timezone_coord'].astype('object')

In [19]:
df['tt_lat'] = df['tweet_location_coord'].apply(lambda x: None if type(x) is float else x['lat'])
df['tt_lng'] = df['tweet_location_coord'].apply(lambda x: None if type(x) is float else x['lng'])

In [20]:
df['usr_lat'] = df['user_timezone_coord'].apply(lambda x: None if type(x) is float else x['lat'])
df['usr_lng'] = df['user_timezone_coord'].apply(lambda x: None if type(x) is float else x['lng'])

In [21]:
df['hour'] = df['tweet_created'].dt.hour
df['date'] = df['tweet_created'].dt.date
df['dow'] = df['tweet_created'].dt.dayofweek

In [25]:
df.head()

,tweet_id,airline_sentiment,is_reply,reply_count,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,...,airline,tweet_location_coord,user_timezone_coord,tt_lat,tt_lng,usr_lat,usr_lng,hour,date,dow
0,942743012337122944,positive,False,0,0,"""Los pilotos de Ryanair desconvocan la huelga ...",nan,2017-12-18 13:07:04,NaN,Dublin,...,NaN,NaN,"{'lat': 53.3498053, 'lng': -6.2603097}",nan,nan,53.349805,-6.260310,13,2017-12-18,0
1,926857871916184064,positive,True,0,0,"@Iberia @lavecinarubia Si ,por favor las decla...",nan,2017-11-04 17:05:11,NaN,NaN,...,Iberia|lavecinarubia,NaN,NaN,nan,nan,nan,nan,17,2017-11-04,5
2,936979305720091008,neutral,True,0,0,@Iberia Me dirías por favor que costo tiene?,nan,2017-12-02 15:24:09,NaN,NaN,...,Iberia,NaN,NaN,nan,nan,nan,nan,15,2017-12-02,5
3,943983853802328064,negative,True,0,0,"@SupermanlopezN @Iberia @giroditalia Champion,...",nan,2017-12-21 23:17:43,NaN,Central Time (US & Canada),...,SupermanlopezN|Iberia|giroditalia,NaN,"{'lat': 37.09024, 'lng': -95.712891}",nan,nan,37.090240,-95.712891,23,2017-12-21,3
4,938207464457212032,negative,True,0,0,@SrtaFarrellDM @KLM @Iberia Eso de avianca es ...,nan,2017-12-06 00:44:25,NaN,Eastern Time (US & Canada),...,SrtaFarrellDM|KLM|Iberia,NaN,"{'lat': 37.51647579999999, 'lng': -82.8067126}",nan,nan,37.516476,-82.806713,0,2017-12-06,2


In [26]:
dfExport = df[['tweet_id','airline_sentiment', 'airline', 'lat', 'lon', 'tweet_location', 'tt_lat', 'tt_lng','user_timezone', 'usr_lat', 'usr_lng','hour','date','dow']]

In [27]:
dfExport.to_csv(path_or_buf=fileOut, sep=',', na_rep='', header=True, index=False, encoding='utf-8', mode='w', quotechar='"', line_terminator='\n', decimal='.')